**Purpose**

*This notebook introduces substantive modeling choices e.g.*
- data aggregation, 
- dropping missing data, 
- and coding more features

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import geopandas as gpd
from datetime import timedelta

from data.constants import DNC_START, DNC_END, LOCAL_CRS, WORLD_CRS
from data.datemath import from_ymd, date_aggs, to_yw
from power import power_reg

In [2]:
point_panel_in = "../data/interim/point_panel.parquet"
line_panel_in = "../data/interim/line_panel.parquet"
tract_panel_in = "../data/interim/tract_panel.parquet"
comm_panel_in = "../data/interim/comm_panel.parquet"
tracts_in = "../data/interim/tracts.geoparquet"

point_panel_out = "../data/final/point_panel.parquet"
line_panel_out = "../data/final/line_panel.parquet"
tract_panel_out = "../data/final/tract_panel.parquet"
comm_panel_out = "../data/final/comm_panel.parquet"

# Pipeline In

In [3]:
point_panel = gpd.read_parquet(point_panel_in)
line_panel = gpd.read_parquet(line_panel_in)
tract_panel = gpd.read_parquet(tract_panel_in)
comm_panel = gpd.read_parquet(comm_panel_in)
tracts = gpd.read_parquet(tracts_in)['geoid10']

# Code Features

In [4]:
point_panel = point_panel.pipe(date_aggs, 'date')
line_panel = line_panel.pipe(date_aggs, 'date')
tract_panel = tract_panel.pipe(date_aggs, 'date')
comm_panel = comm_panel.pipe(date_aggs, 'date')

In [5]:
# Substantive!
# There aren't enough data points within 400m or even 800m.
# In panel.ipynb we show that UC and MP have higher ridership than other
# places, and furthermore that the signal is stronger closer to UC and MP
# and reverts to the mean as the buffer size grows.
# Thus, using the largest buffer size attenuates and possibly confounds our
# signal with irrelevant rides, but is necessary to achieve a minimal sample
# of treated units.

def label_ucmp(df):
    df['UCMP'] = np.where((df['uc_1600'] > 0) | (df['mp_1600'] > 0), 1,0)
    return df.drop(columns=df.filter(regex='uc_|mp_').columns)

point_panel = point_panel.pipe(label_ucmp)
line_panel = line_panel.pipe(label_ucmp)
tract_panel = tract_panel.pipe(label_ucmp)
comm_panel = comm_panel.pipe(label_ucmp)

In [6]:
def fillna(df, fills: dict):
    return df.assign(**{col: df[col].fillna(val) for col, val in fills.items()})

# Assert that null features are due to non-chicago tracts. 
# Let's impute correct values instead of dropping tracts outside of chicago (ie ubers and bike stations)
assert all(tract_panel['UCMP'].notna() | ~tract_panel['id'].isin(tracts))
assert all(tract_panel['airport'].notna() | ~tract_panel['id'].isin(tracts))
assert all(point_panel[['UCMP','airport']].notna().all(axis=1) |
         ((point_panel['transit']=='uber') & point_panel['tract'].notna()))

point_panel = point_panel.pipe(fillna, {'UCMP': 0, 'airport': 0})
line_panel = line_panel.pipe(fillna, {'UCMP': 0, 'airport': 0})
tract_panel = tract_panel.pipe(fillna, {'UCMP': 0, 'airport': 0})
comm_panel = comm_panel.pipe(fillna, {'UCMP': 0, 'airport': 0})

In [7]:
def label_latlong(df):
    # Let's standardize lat/long because they are measured in millions of feet
    # So doing so helps keep the relative scale of X vs Y in check. 
    scale = lambda x: (x - x.mean()) / x.std()
    df['lat'] = scale(df.geometry.to_crs(LOCAL_CRS).centroid.to_crs(WORLD_CRS).y)
    df['long'] = scale(df.geometry.to_crs(LOCAL_CRS).centroid.to_crs(WORLD_CRS).x)
    return df

point_panel = point_panel.pipe(label_latlong)
line_panel = line_panel.pipe(label_latlong)
tract_panel = tract_panel.pipe(label_latlong)
comm_panel = comm_panel.pipe(label_latlong)

In [8]:
def binarize(df, col):
    return df.assign(**{col: df[col]*1.0})

point_panel = point_panel.pipe(binarize, 'DNC')
line_panel = line_panel.pipe(binarize, 'DNC')
tract_panel = tract_panel.pipe(binarize, 'DNC')
comm_panel = comm_panel.pipe(binarize, 'DNC')

# Drop Data

## Out of Bounds Bike Stations

In [9]:
# XXX:
# Dropping all nan columns FOR NOW because I don't feel like dealing with it.
# We can probably fix it all by getting all cook county census tracts instead of just chicago.

print("Dropping {} ({:.0%}) point rows".format(point_panel.isna().any(axis=1).sum(),point_panel.isna().any(axis=1).mean()))
print("Dropping {} ({:.0%}) line rows".format(line_panel.isna().any(axis=1).sum(),line_panel.isna().any(axis=1).mean()))
print("Dropping {} ({:.0%}) tract rows".format(tract_panel.isna().any(axis=1).sum(),tract_panel.isna().any(axis=1).mean()))
print("Dropping {} ({:.0%}) comm rows".format(comm_panel.isna().any(axis=1).sum(),comm_panel.isna().any(axis=1).mean()))
point_panel = point_panel.dropna()
line_panel = line_panel.dropna()
tract_panel = tract_panel.dropna()
comm_panel = comm_panel.dropna()

Dropping 7991 (4%) point rows
Dropping 240 (0%) line rows
Dropping 3521 (1%) tract rows
Dropping 0 (0%) comm rows


In [10]:
# XXX: Substantive!! 
#      Taking all weeks in June, July, August
MODEL_PRE_WEEKS = 11
MODEL_POST_WEEKS = 1

model_start_yearweek = to_yw(from_ymd(DNC_START) - timedelta(weeks=MODEL_PRE_WEEKS))
model_end_yearweek = to_yw(from_ymd(DNC_END) + timedelta(weeks=MODEL_POST_WEEKS))

point_panel = point_panel[point_panel['year-week'].between(model_start_yearweek, model_end_yearweek)]
line_panel = line_panel[line_panel['year-week'].between(model_start_yearweek, model_end_yearweek)]
tract_panel = tract_panel[tract_panel['year-week'].between(model_start_yearweek, model_end_yearweek)]
comm_panel = comm_panel[comm_panel['year-week'].between(model_start_yearweek, model_end_yearweek)]

# Aggregate over Time

In [11]:
def model_agg(df):
    unit_aggs = {k: 'first' for k in ['UCMP','airport','lat','long']}
    time_aggs = {'rides':'sum','DNC':'max', 'is_weekend': 'max'}  
    # weekend is constant for weekly model but we just won't include it in the model spec
    df_weekly = df.groupby(['year-week','id','transit'], as_index=False).agg(unit_aggs | time_aggs)
    df_daily = df.groupby(['date','id','transit'], as_index=False).agg(unit_aggs | time_aggs)
    return df_weekly, df_daily

point_panel_weekly, point_panel_daily = model_agg(point_panel)
line_panel_weekly, line_panel_daily = model_agg(line_panel)
tract_panel_weekly, tract_panel_daily = model_agg(tract_panel)
comm_panel_weekly, comm_panel_daily = model_agg(comm_panel)

# Choose and Justify Agg

Monthly: obviously stupid. attenuates treatment too much.

Weekly: we have 4/7 days of the week in the DNC. and no weekend days actually.
so we attenuate our signal by 3/7. 

Daily: we have more trouble modeling dotw variation. (maybe not an issue after eliminating weekends)

## with non-treatment model

In [12]:
def partial_r2(full, reduced, full_name, reduced_name):
    better = max(full.rsquared, reduced.rsquared)
    worse = min(full.rsquared, reduced.rsquared)
    partial = (better - worse) / (1 - worse)  # Proportion of unexplained variance explained by better model.
    better_name = full_name if full.rsquared > reduced.rsquared else reduced_name
    print("better model = {}, R2 = {:.3f}, partial R2 = {:.3f}".format(better_name, better, partial))

In [13]:
# Compute the partial R2 of weekly vs daily. Only include non-treatment regressors.
formula = "np.log1p(rides) ~ transit + (lat + long)**2 + I(lat**2) + I(long**2)"
full = sm.OLS.from_formula(formula, point_panel_daily).fit()
reduced = sm.OLS.from_formula(formula, point_panel_weekly).fit()
partial_r2(full, reduced, "daily", "weekly")
# The weekly model actually has a slightly higher R2, 
# meaning we're not great at explaining DOTW variation.

better model = daily, R2 = 0.771, partial R2 = 0.072


In [14]:
# Power analysis:
DNC_ATTEND = 5e4
effect_sum = 4 * 2 * DNC_ATTEND  # days in event * rides per day * attendees

pd.concat([power_reg(full, np.log1p(effect_sum/len(point_panel_daily))).rename('daily'),
           power_reg(reduced, np.log1p(effect_sum/len(point_panel_weekly))).rename('weekly')],axis=1) \
        .assign(ratio=lambda x: x['weekly']/x['daily'])
# The daily model has smaller residual variance but identical power achieved.

,daily,weekly,ratio
mean,3.440143,4.201957,1.221448
std,1.025104,1.324865,1.292419
mde,0.009887,0.028959,2.928939
effect_size,1.747618,3.232726,1.84979
power_achieved,1.0,1.0,1.0
t_stat,495.19644,312.744557,0.631557
is_detectable,True,True,1.0


## with treatment model

In [15]:
# Compute the partial R2 of weekly vs daily. 
formula = "np.log1p(rides) ~ DNC * transit + (lat + long)**2 + I(lat**2) + I(long**2)"
full = sm.OLS.from_formula(formula, point_panel_daily).fit()
reduced = sm.OLS.from_formula(formula, point_panel_weekly).fit()
partial_r2(full, reduced, "daily", "weekly")
# The weekly model actually has a slightly higher R2, 
# meaning we're not great at explaining DOTW variation.

better model = daily, R2 = 0.771, partial R2 = 0.071


In [16]:
pd.concat([power_reg(full, np.log1p(effect_sum/len(point_panel_daily)), "DNC").rename('daily'),
           power_reg(reduced, np.log1p(effect_sum/len(point_panel_weekly)), "DNC").rename('weekly')],axis=1) 
# The standard errors are smaller on the daily model but achieved effect size is net the same,
# probably because we didn't change the effect delta but scaled down the nobs and effect size
# by the same amount.

,daily,weekly
mean,0.0,0.0
std,0.018314,0.042378
mde,0.000177,0.000926
effect_size,1.747618,3.232726
power_achieved,1.0,1.0
t_stat,27717.774516,9777.326135
is_detectable,True,True


I think we'll keep the daily observations because it gives us more flexibility to 
switch to alternative treatment specifications (buffer or event study) later on.

# Pipeline Out

In [17]:
point_panel_daily.to_parquet(point_panel_out, index=False)
line_panel_daily.to_parquet(line_panel_out, index=False)
tract_panel_daily.to_parquet(tract_panel_out, index=False)
comm_panel_daily.to_parquet(comm_panel_out, index=False)